In [ ]:
pip install vk_api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 1.6 MB/s eta 0:00:00


In [ ]:
#@title Текст заголовка по умолчанию
import vk_api
import csv
from datetime import datetime, timedelta

access_token = 'vk1.a.jWgYkRybRbNa0_fUYXI54921-'
group_id = 80420653

# Создание сессии VK API и авторизация
vk_session = vk_api.VkApi(token=access_token)
vk = vk_session.get_api()

def get_all_community_messages(group_id):
    all_messages = []  # Список для хранения всех входящих сообщений

    # Получение всех диалогов сообщества
    conversations = vk.messages.getConversations(group_id=group_id, count=200)['items']

    # Обработка диалогов
    for conversation in conversations:
        conversation_id = conversation['conversation']['peer']['id']
        conversation_type = conversation['conversation']['peer']['type']

        # Получение информации о пользователе
        if conversation_type == 'user':
            user_id = conversation['conversation']['peer']['id']
            user_info = vk.users.get(user_ids=user_id)[0]
            user_name = user_info['first_name'] + ' ' + user_info['last_name']
        else:
            user_name = 'Сообщество'

        # Получение всех сообщений в текущем диалоге
        messages = vk.messages.getHistory(peer_id=conversation_id, count=200)['items']

        # Обработка сообщений
        for message in messages:
            if 'from_id' in message and message['from_id'] != group_id:  # Проверка на входящее сообщение
                message_id = message['id']
                date = message['date']
                text = message['text']

                # Преобразование значения даты и времени в понятный формат
                datetime_obj = datetime.fromtimestamp(date)
                formatted_datetime = datetime_obj.strftime('%Y-%m-%d %H:%M:%S')

                # Добавление сообщения в список всех входящих сообщений
                all_messages.append({
                    'message_id': message_id,
                    'text': text,
                    'user_name': user_name,
                    'date': formatted_datetime
                })

    # Сохранение данных в CSV файл
    with open('messages.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
        fieldnames = ['ID сообщения', 'Текст', 'Пользователь (Имя)', 'Дата и время']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')
        writer.writeheader()
        for message in all_messages:
            writer.writerow({
                'ID сообщения': message['message_id'],
                'Текст': message['text'],
                'Пользователь (Имя)': message['user_name'],
                'Дата и время': message['date']
            })

get_all_community_messages(group_id)
